In [14]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Metrics
from sklearn.metrics import f1_score, classification_report

# Imbalanced learning - IMPROVED
from imblearn.over_sampling import ADASYN  # Better than SMOTE!
from imblearn.combine import SMOTETomek

import optuna
from optuna.samplers import TPESampler

print("=" * 80)
print("PLAYER SEGMENT CLASSIFICATION PIPELINE")
print("=" * 80)

PLAYER SEGMENT CLASSIFICATION PIPELINE


1. LOAD AND PREPROCESS DATA (Same as main pipeline)

In [3]:
print("\n[1] Loading and Preprocessing Data...")

train = pd.read_csv('C:\\Users\\Kanyavan\\Documents\\Year3_semester1\\ML\\CPE342-Hackathon\\task2\\train.csv')
train.info()


[1] Loading and Preprocessing Data...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101658 entries, 0 to 101657
Data columns (total 47 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                101658 non-null  object 
 1   player_id                         101658 non-null  object 
 2   play_frequency                    93785 non-null   float64
 3   avg_session_duration              93771 non-null   float64
 4   total_playtime_hours              93887 non-null   float64
 5   login_streak                      93938 non-null   float64
 6   days_since_last_login             93878 non-null   float64
 7   total_spending_thb                93902 non-null   float64
 8   avg_monthly_spending              93923 non-null   float64
 9   spending_frequency                93898 non-null   float64
 10  friend_count                      93720 non-null   float64
 11  team_play_per

In [4]:
print(f"Train shape: {train.shape}")
print(f"Target distribution:\n{train['segment'].value_counts().sort_index()}")

Train shape: (101658, 47)
Target distribution:
segment
0    40064
1    25397
2    20549
3    15648
Name: count, dtype: int64


In [5]:
X_train = train.drop(['id', 'player_id', 'segment'], axis=1)
y_train = train['segment'].astype(int)

In [ ]:
#-------------------------------------------------FIRST-----------------------------------------------------------------------------
# Feature Engineering (simplified version - use full version from main script)
def engineer_features(df):
    df = df.copy()

    # Spending features
    df['spending_per_hour'] = df['total_spending_thb'] / (df['total_playtime_hours'] + 1)
    df['spending_intensity'] = df['avg_monthly_spending'] * df['spending_frequency']
    df['is_spender'] = (df['total_spending_thb'] > 0).astype(int)

    # Engagement features
    df['engagement_score'] = (df['play_frequency'] * df['avg_session_duration'] * df['login_streak']).fillna(0)
    df['activity_consistency'] = df['login_streak'] / (df['days_since_last_login'] + 1)

    # Social features
    df['social_score'] = (df['friend_count'] * df['team_play_percentage'] * df['chat_activity_score']).fillna(0)
    df['is_social_player'] = (df['team_play_percentage'] > 50).astype(int)

    # Competitive features
    df['competitive_score'] = (df['ranked_participation_rate'] * df['tournament_entries'] * df['competitive_rank']).fillna(0)
    df['is_competitive'] = (df['ranked_participation_rate'] > 50).astype(int)

    # Remove random metrics
    df = df.drop(['random_metric_1', 'random_metric_2', 'random_metric_3'], axis=1, errors='ignore')

    return df

X_train_eng = engineer_features(X_train)
#-------------------------------------------------FIRST-----------------------------------------------------------------------------

In [ ]:
#-------------------------------------------------FIRST-----------------------------------------------------------------------------
# Preprocessing
numerical_features = X_train_eng.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train_eng.select_dtypes(include=['object']).columns.tolist()
#-------------------------------------------------FIRST-----------------------------------------------------------------------------

In [12]:
# Label Encoding
for col in categorical_features:
    le = LabelEncoder()
    X_train_eng[col] = le.fit_transform(X_train_eng[col].astype(str))

In [13]:
# Imputation
num_imputer = SimpleImputer(strategy='median')
X_train_eng[numerical_features] = num_imputer.fit_transform(X_train_eng[numerical_features])

if categorical_features:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    X_train_eng[categorical_features] = cat_imputer.fit_transform(X_train_eng[categorical_features])

In [14]:
# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_eng)

In [15]:
# SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

print(f"Data prepared: {X_train_balanced.shape}")

Data prepared: (160256, 50)


2. OPTUNA HYPERPARAMETER TUNING

In [16]:
# Define cross-validation strategy
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

2.1 XGBoost Hyperparameter Tuning

In [17]:
def objective_xgb(trial):
    """Objective function for XGBoost optimization"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'random_state': 42,
        'eval_metric': 'mlogloss',
        'tree_method': 'hist'
    }

    model = XGBClassifier(**params)
    scores = cross_val_score(model, X_train_balanced, y_train_balanced,
                            cv=skf, scoring='f1_macro', n_jobs=2)

    return scores.mean()

In [18]:
# Run optimization
study_xgb = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42),
    study_name='XGBoost_Optimization'
)

study_xgb.optimize(objective_xgb, n_trials=20, show_progress_bar=True)

print(f"\nBest XGBoost F1-Score: {study_xgb.best_value:.4f}")
print("Best XGBoost Parameters:")
for key, value in study_xgb.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-20 07:16:19,931] A new study created in memory with name: XGBoost_Optimization
Best trial: 0. Best value: 0.811537:   5%|▌         | 1/20 [01:19<25:08, 79.41s/it]

[I 2025-11-20 07:17:39,345] Trial 0 finished with value: 0.81153703799887 and parameters: {'n_estimators': 250, 'max_depth': 8, 'learning_rate': 0.1205712628744377, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'reg_alpha': 8.661761457749352, 'reg_lambda': 6.011150117432088}. Best is trial 0 with value: 0.81153703799887.


Best trial: 0. Best value: 0.811537:  10%|█         | 2/20 [02:06<18:03, 60.17s/it]

[I 2025-11-20 07:18:26,042] Trial 1 finished with value: 0.7964559307964221 and parameters: {'n_estimators': 383, 'max_depth': 3, 'learning_rate': 0.2708160864249968, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'min_child_weight': 2, 'gamma': 0.9170225492671691, 'reg_alpha': 3.0424224295953772, 'reg_lambda': 5.247564316322379}. Best is trial 0 with value: 0.81153703799887.


Best trial: 0. Best value: 0.811537:  15%|█▌        | 3/20 [02:47<14:35, 51.50s/it]

...

[I 2025-11-20 07:32:55,758] Trial 17 finished with value: 0.7925049805739734 and parameters: {'n_estimators': 110, 'max_depth': 6, 'learning_rate': 0.14290274997706512, 'subsample': 0.9285086088875721, 'colsample_bytree': 0.8505915654206465, 'min_child_weight': 6, 'gamma': 0.6234135690000955, 'reg_alpha': 4.916273921024845, 'reg_lambda': 7.165494609738724}. Best is trial 0 with value: 0.81153703799887.


Best trial: 18. Best value: 0.81231:  95%|█████████▌| 19/20 [17:23<00:39, 39.95s/it]

[I 2025-11-20 07:33:43,779] Trial 18 finished with value: 0.8123102421326547 and parameters: {'n_estimators': 214, 'max_depth': 8, 'learning_rate': 0.2810657274670111, 'subsample': 0.6026168324121943, 'colsample_bytree': 0.6075934324519439, 'min_child_weight': 5, 'gamma': 0.553617064887681, 'reg_alpha': 8.157298373095427, 'reg_lambda': 3.7834031171882017}. Best is trial 18 with value: 0.8123102421326547.


Best trial: 18. Best value: 0.81231: 100%|██████████| 20/20 [18:15<00:00, 54.79s/it]

[I 2025-11-20 07:34:35,820] Trial 19 finished with value: 0.7914299999384486 and parameters: {'n_estimators': 212, 'max_depth': 8, 'learning_rate': 0.03910891361220486, 'subsample': 0.6172803398163924, 'colsample_bytree': 0.6216495646994378, 'min_child_weight': 3, 'gamma': 0.471058024403758, 'reg_alpha': 9.984912975919531, 'reg_lambda': 0.43357095923358635}. Best is trial 18 with value: 0.8123102421326547.

Best XGBoost F1-Score: 0.8123
Best XGBoost Parameters:
  n_estimators: 214
  max_depth: 8
  learning_rate: 0.2810657274670111
  subsample: 0.6026168324121943
  colsample_bytree: 0.6075934324519439
  min_child_weight: 5
  gamma: 0.553617064887681
  reg_alpha: 8.157298373095427
  reg_lambda: 3.7834031171882017


2.2 LightGBM Hyperparameter Tuning

In [19]:
print("\n[2.2] Tuning LightGBM Hyperparameters...")


[2.2] Tuning LightGBM Hyperparameters...


In [20]:
def objective_lgbm(trial):
    """Objective function for LightGBM optimization"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'random_state': 42,
        'verbose': -1
    }

    model = LGBMClassifier(**params)
    scores = cross_val_score(model, X_train_balanced, y_train_balanced,
                            cv=skf, scoring='f1_macro', n_jobs=2)

    return scores.mean()

In [21]:
# Run optimization
study_lgbm = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42),
    study_name='LightGBM_Optimization'
)

study_lgbm.optimize(objective_lgbm, n_trials=20, show_progress_bar=True)

print(f"\nBest LightGBM F1-Score: {study_lgbm.best_value:.4f}")
print("Best LightGBM Parameters:")
for key, value in study_lgbm.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-20 07:41:11,201] A new study created in memory with name: LightGBM_Optimization
Best trial: 0. Best value: 0.803279:   5%|▌         | 1/20 [00:29<09:16, 29.30s/it]

[I 2025-11-20 07:41:40,501] Trial 0 finished with value: 0.8032788344805379 and parameters: {'n_estimators': 250, 'max_depth': 8, 'learning_rate': 0.1205712628744377, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'min_child_samples': 19, 'num_leaves': 30, 'reg_alpha': 8.661761457749352, 'reg_lambda': 6.011150117432088}. Best is trial 0 with value: 0.8032788344805379.


Best trial: 0. Best value: 0.803279:  10%|█         | 2/20 [00:51<07:31, 25.08s/it]

[I 2025-11-20 07:42:02,629] Trial 1 finished with value: 0.7984306415850317 and parameters: {'n_estimators': 383, 'max_depth': 3, 'learning_rate': 0.2708160864249968, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'min_child_samples': 22, 'num_leaves': 53, 'reg_alpha': 3.0424224295953772, 'reg_lambda': 5.247564316322379}. Best is trial 0 with value: 0.8032788344805379.


Best trial: 0. Best value: 0.803279:  15%|█▌        | 3/20 [01:11<06:30, 22.96s/it]

...

[I 2025-11-20 07:49:02,306] Trial 17 finished with value: 0.7772542229730912 and parameters: {'n_estimators': 171, 'max_depth': 5, 'learning_rate': 0.03721349294209224, 'subsample': 0.9579190418236523, 'colsample_bytree': 0.8758697700847069, 'min_child_samples': 49, 'num_leaves': 118, 'reg_alpha': 4.026609486224885, 'reg_lambda': 6.946016703174446}. Best is trial 15 with value: 0.8111242718020497.


Best trial: 15. Best value: 0.811124:  95%|█████████▌| 19/20 [08:18<00:24, 24.63s/it]

[I 2025-11-20 07:49:29,394] Trial 18 finished with value: 0.8106399626475481 and parameters: {'n_estimators': 182, 'max_depth': 7, 'learning_rate': 0.15000385515031378, 'subsample': 0.9299305629337544, 'colsample_bytree': 0.7602802489247009, 'min_child_samples': 67, 'num_leaves': 164, 'reg_alpha': 6.435468714317403, 'reg_lambda': 7.1058773596334}. Best is trial 15 with value: 0.8111242718020497.


Best trial: 15. Best value: 0.811124: 100%|██████████| 20/20 [08:39<00:00, 25.99s/it]

[I 2025-11-20 07:49:50,917] Trial 19 finished with value: 0.784390637487131 and parameters: {'n_estimators': 209, 'max_depth': 5, 'learning_rate': 0.03910891361220486, 'subsample': 0.9394691950251458, 'colsample_bytree': 0.7640881690357239, 'min_child_samples': 86, 'num_leaves': 169, 'reg_alpha': 6.538839629254709, 'reg_lambda': 0.43357095923358635}. Best is trial 15 with value: 0.8111242718020497.

Best LightGBM F1-Score: 0.8111
Best LightGBM Parameters:
  n_estimators: 176
  max_depth: 7
  learning_rate: 0.14920643617808255
  subsample: 0.9750244637464088
  colsample_bytree: 0.8516529430033545
  min_child_samples: 55
  num_leaves: 123
  reg_alpha: 4.257738884256797
  reg_lambda: 7.202107114363317


2.3 CatBoost Hyperparameter Tuning

In [22]:
print("\n[2.3] Tuning CatBoost Hyperparameters...")


[2.3] Tuning CatBoost Hyperparameters...


In [23]:
def objective_catboost(trial):
    """Objective function for CatBoost optimization"""
    params = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'depth': trial.suggest_int('depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'random_state': 42,
        'verbose': False
    }

    model = CatBoostClassifier(**params)
    scores = cross_val_score(model, X_train_balanced, y_train_balanced,
                            cv=skf, scoring='f1_macro', n_jobs=2)

    return scores.mean()


In [25]:
# Run optimization
study_catboost = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42),
    study_name='CatBoost_Optimization'
)

study_catboost.optimize(objective_catboost, n_trials=20, show_progress_bar=True)

print(f"\nBest CatBoost F1-Score: {study_catboost.best_value:.4f}")
print("Best CatBoost Parameters:")
for key, value in study_catboost.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-20 08:01:28,827] A new study created in memory with name: CatBoost_Optimization
Best trial: 0. Best value: 0.796945:   5%|▌         | 1/20 [02:11<41:45, 131.86s/it]

[I 2025-11-20 08:03:40,691] Trial 0 finished with value: 0.79694460970667 and parameters: {'iterations': 250, 'depth': 8, 'learning_rate': 0.1205712628744377, 'l2_leaf_reg': 6.387926357773329, 'border_count': 66, 'random_strength': 1.5599452033620265, 'bagging_temperature': 0.05808361216819946}. Best is trial 0 with value: 0.79694460970667.


Best trial: 1. Best value: 0.797855:  10%|█         | 2/20 [03:52<34:01, 113.44s/it]

[I 2025-11-20 08:05:21,232] Trial 1 finished with value: 0.797855159973342 and parameters: {'iterations': 447, 'depth': 6, 'learning_rate': 0.11114989443094977, 'l2_leaf_reg': 1.185260448662222, 'border_count': 249, 'random_strength': 8.324426408004218, 'bagging_temperature': 0.21233911067827616}. Best is trial 1 with value: 0.797855159973342.


Best trial: 1. Best value: 0.797855:  15%|█▌        | 3/20 [04:24<21:38, 76.36s/it] 

...

[I 2025-11-20 08:53:05,246] Trial 17 finished with value: 0.783511441912777 and parameters: {'iterations': 443, 'depth': 5, 'learning_rate': 0.04848754515310091, 'l2_leaf_reg': 4.747699312717497, 'border_count': 151, 'random_strength': 6.4931800423011925, 'bagging_temperature': 0.1797798214520778}. Best is trial 13 with value: 0.8044206198911465.


Best trial: 13. Best value: 0.804421:  95%|█████████▌| 19/20 [52:37<02:42, 162.33s/it]

[I 2025-11-20 08:54:06,142] Trial 18 finished with value: 0.7882312439750974 and parameters: {'iterations': 317, 'depth': 5, 'learning_rate': 0.08673832921866055, 'l2_leaf_reg': 2.1940896313268574, 'border_count': 198, 'random_strength': 3.8876919478638623, 'bagging_temperature': 0.3944294945377801}. Best is trial 13 with value: 0.8044206198911465.


Best trial: 13. Best value: 0.804421: 100%|██████████| 20/20 [54:28<00:00, 163.44s/it]

[I 2025-11-20 08:55:57,571] Trial 19 finished with value: 0.8008523908768035 and parameters: {'iterations': 387, 'depth': 7, 'learning_rate': 0.16299485419239088, 'l2_leaf_reg': 8.66365647571051, 'border_count': 184, 'random_strength': 8.512079589812016, 'bagging_temperature': 0.1527105193948213}. Best is trial 13 with value: 0.8044206198911465.

Best CatBoost F1-Score: 0.8044
Best CatBoost Parameters:
  iterations: 376
  depth: 7
  learning_rate: 0.2599957906944455
  l2_leaf_reg: 3.3122077050601906
  border_count: 157
  random_strength: 9.919547750113695
  bagging_temperature: 0.22713901165087858


3. SAVE BEST PARAMETERS

In [26]:
print("\n[3] Saving Best Parameters...")


[3] Saving Best Parameters...


In [27]:
best_params = {
    'XGBoost': study_xgb.best_params,
    'LightGBM': study_lgbm.best_params,
    'CatBoost': study_catboost.best_params
}

best_scores = {
    'XGBoost': study_xgb.best_value,
    'LightGBM': study_lgbm.best_value,
    'CatBoost': study_catboost.best_value
}

# Save to file
import json

with open('best_hyperparameters.json', 'w') as f:
    json.dump(best_params, f, indent=4)

print("\nBest parameters saved to 'best_hyperparameters.json'")


Best parameters saved to 'best_hyperparameters.json'


4. SUMMARY

In [28]:
print("\n" + "=" * 80)
print("HYPERPARAMETER TUNING SUMMARY")
print("=" * 80)

for model_name, score in best_scores.items():
    print(f"{model_name:20s} | Best F1-Score: {score:.4f}")

best_model = max(best_scores, key=best_scores.get)
print(f"\nBest Overall Model: {best_model} with F1-Score: {best_scores[best_model]:.4f}")

print("\n" + "=" * 80)
print("INSTRUCTIONS:")
print("=" * 80)
print("1. Use the best parameters from 'best_hyperparameters.json'")
print("2. Update the main pipeline with these optimized parameters")
print("3. Retrain models and generate final predictions")
print("=" * 80)


HYPERPARAMETER TUNING SUMMARY
XGBoost              | Best F1-Score: 0.8123
LightGBM             | Best F1-Score: 0.8111
CatBoost             | Best F1-Score: 0.8044

Best Overall Model: XGBoost with F1-Score: 0.8123

INSTRUCTIONS:
1. Use the best parameters from 'best_hyperparameters.json'
2. Update the main pipeline with these optimized parameters
3. Retrain models and generate final predictions


5. TRAIN FINAL MODELS WITH BEST PARAMETERS

In [29]:
print("\n[5] Training Final Models with Best Parameters...")




[5] Training Final Models with Best Parameters...


In [30]:
# Train final models
final_models = {}

# XGBoost
xgb_params = study_xgb.best_params.copy()
xgb_params.update({'random_state': 42, 'eval_metric': 'mlogloss', 'tree_method': 'hist'})
final_models['XGBoost'] = XGBClassifier(**xgb_params)
final_models['XGBoost'].fit(X_train_balanced, y_train_balanced)

# LightGBM
lgbm_params = study_lgbm.best_params.copy()
lgbm_params.update({'random_state': 42, 'verbose': -1})
final_models['LightGBM'] = LGBMClassifier(**lgbm_params)
final_models['LightGBM'].fit(X_train_balanced, y_train_balanced)

# CatBoost
cat_params = study_catboost.best_params.copy()
cat_params.update({'random_state': 42, 'verbose': False})
final_models['CatBoost'] = CatBoostClassifier(**cat_params)
final_models['CatBoost'].fit(X_train_balanced, y_train_balanced)

print("\nFinal models trained successfully!")
print("You can now use these models in the main pipeline for predictions.")

# Save models (optional)
import pickle

for model_name, model in final_models.items():
    with open(f'model_{model_name.lower()}_optimized.pkl', 'wb') as f:
        pickle.dump(model, f)
    print(f"Saved: model_{model_name.lower()}_optimized.pkl")

print("\n" + "=" * 80)
print("HYPERPARAMETER TUNING COMPLETED!")
print("=" * 80)


Final models trained successfully!
You can now use these models in the main pipeline for predictions.
Saved: model_xgboost_optimized.pkl
Saved: model_lightgbm_optimized.pkl
Saved: model_catboost_optimized.pkl

HYPERPARAMETER TUNING COMPLETED!
